# Install Dependencies

In [ ]:
%pip install psycopg2
%pip install sqlalchemy

# Create the database

In [2]:
import psycopg2
from sqlalchemy import create_engine

# Function to create a new database
def create_database(db_name, username, password, host='localhost', port=5432):
    conn = psycopg2.connect(dbname="postgres", user=username, password=password, host=host, port=port)
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    cur.execute(f"CREATE DATABASE {db_name};")
    cur.close()
    conn.close()

# Create the database
create_database('angeliteforecast', 'postgres', 'thesis')

DuplicateDatabase: database "angeliteforecast" already exists


# Connect to the database

In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

def connect_to_database(username, password, host='34.126.160.94', port=5432, db_name='angeliteforecast'):
    """Create a database connection."""
    connection_string = f'postgresql://{username}:{password}@{host}:{port}/{db_name}'
    engine = create_engine(connection_string)
    return engine
engine = connect_to_database("postgres", "thesis")



# Data Cleaning

In [3]:
import pandas as pd


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

similar_majors_dict = {
    # CCJEF
    "Criminology": ["BS Criminology"], # Combine Criminology and BS Criminology

    
    # SAS
    "ABComm": ["BA Comm"],
    "CommArts": ["BA Comm"],
    "AdvPublicRel": ["BA Comm"], # Area of Specialization
    # "CommArts": "BA Comm",     # Lack of Data | Drop


    # SBA
    "Accounting": ["Accountancy"],
    "Accounting Tech": ["BSAcctgInfoSys"],
    "BSBA-BM-Marketi": ["BusMgt", "BSBA-MktgMgmt"],
    "BussMgmt-HRM": ["BusMgt", "BSBA-HRMgmt"],


    # SEA
    "ECE": ["ELECENG"],
    # "BSCE-CEM": ["CE", "BSCE-CEM"],
    # "BSCE-SE": ["CE", "BSCE-SE"],
    # "BSCE-TE": ["CE", "BSCE-TE"],




    # SED
    "BEED-MajSPED": ["BSNEd"],           # Name change?
    "BEED-SpecialEdu": ["BSNEd"],        # Name change?
    "BPE-SPE": ["BPEd"],                   # Name change?
    "BSED-ValEd": ["BSED-RelValEd"],
    "BSMath": ["Math"],
    

    # SHTM
    "BS EventMgmt": ["BSTM-Events"],


    # SOC
    "BSCSSysDev": ["BSCompsci"],              # New Curriculum
    "BSITAnimation": ["BSEMC-DA"],            # New Curriculum
    "BSITAreaAnimati": ["BSEMC-DA"],          # New Curriculum
    "BSCyberplusPSM": ["BSCybersecurity"],    # Same Major but with Professional Science Master's Degree
    # "BSInfoTech": "BSIT",                 # General IT Major | Lack of Data | Drop
    # "BSITMultiTec": "BSIT",               # Discontinued | Lack of Data | Drop
    "BSITAreaNetAdmi": ["BSITNetAdmin"], 
    "BSITAreaWebDev": ["BSITWebdev"],

    }

# Dictionary for major with incorrect department
incorrect_department_dict = {
    "BSBA-HRM": "SBA",
    # "MAPEH-BSED": "SED",
}

department_dict = {
    "CHTM" : "SHTM",
    "CICT" : "SOC",
}

# List of majors to drop
drop_majors = [
    # MA
    # "MAPEH-BSED",     # Lack of data | Discontinued | SED Department?
    "MBM",              # Lack of data | only 1 student
    "MSEngMgmt",        # Lack of data | only 1 student
    

    # SAS
    # "CommArts",         # Lack of data | only 1 student
    "LanguageLit",      # Lack of data | only 1 student

    # SED
    
    # SHTM
    "BSBATourism",
    "BSTM-Tourism",     # Lack of data | BSTourism? | only 2 students


    # SOC
    "AssCompSci",       # Lack of data | only 1 student
    "BSITMultiTec",     # Lack of data | Discontinued
    "BSInfoTech",       # Lack of data | only 1 student

    # SEA
    "ECETech.",        # Lack of data | Discontinued


    # Extras | Aggregate Counts
    "TOTAL",
    "GRAND TOTAL",
]



In [20]:
def clean_data(df):
    

    # Step 2: Correct departments

    for major, department in incorrect_department_dict.items():
        df.loc[df["Major"]== major, 'Department'] = department
        
    # Step 3: Rename departments
    df['Department'] = df['Department'].replace(department_dict)
    # Step 4: If duplicate, get sum
    numerical_features = df.loc[:, "1st_Year":].columns
    df = df.groupby(['Start_Year', 'Semester', 'Department', 'Major'], as_index=False).agg({feature: 'sum' for feature in numerical_features})
    # Step 4: Drop unwanted majors
    # df = df[~df['Major'].isin(drop_majors)]

    # df = df[~df['Department'].isin(['GS', 'JHS', 'HAUSPELL', 'HAU', 'MA'])]
    df = df.reset_index(drop=True)
    df = df.sort_values(by=["Start_Year", "Semester", "Department", "Major"])
    return df


In [26]:
df = pd.read_csv("data/Enrollment_Data.csv").sort_values(by=["Start_Year", "Semester", "Department", "Major"])
cpi_df = pd.read_csv("data/CPI_Education.csv").sort_values(by=["Year", "Month"])
inflation_df = pd.read_csv("data/Inflation_Rate.csv").sort_values(by="Start_Year")
admission_df = pd.read_csv("data/Admission_Data.csv").sort_values(by=["Start_Year", "Department"])
admission_df["Department"] = admission_df["Department"].replace("CICT", "SOC")

hfce_df = pd.read_csv("data/HFCE.csv").sort_values(by=["Start_Year", "Quarter"])
print(df[(df["Department"] == "SOC") & (df["Major"] == "TOTAL")])

df = clean_data(df)
print(df[(df["Department"] == "SOC") & (df["Major"] == "TOTAL")])

# Upload data to database
df.to_sql('enrollment', engine, if_exists='replace', index=False)
cpi_df.to_sql('cpi_education', engine, if_exists='replace', index=False)
inflation_df.to_sql('inflation_rate', engine, if_exists='replace', index=False)
admission_df.to_sql('admission', engine, if_exists='replace', index=False)
hfce_df.to_sql('hfce', engine, if_exists='replace', index=False)


      Major Department  Semester  Start_Year  End_Year  1st_Year  2nd_Year  \
67    TOTAL        SOC         1        2016      2017      76.0     456.0   
134   TOTAL        SOC         2        2016      2017      25.0     431.0   
207   TOTAL        SOC         1        2017      2018       7.0      89.0   
275   TOTAL        SOC         2        2017      2018       3.0      49.0   
352   TOTAL        SOC         1        2018      2019     244.0      26.0   
421   TOTAL        SOC         2        2018      2019     238.0      11.0   
494   TOTAL        SOC         1        2019      2020     326.0     220.0   
561   TOTAL        SOC         2        2019      2020     298.0     216.0   
632   TOTAL        SOC         1        2020      2021     216.0     256.0   
697   TOTAL        SOC         2        2020      2021     196.0     222.0   
773   TOTAL        SOC         1        2021      2022     324.0     219.0   
842   TOTAL        SOC         2        2021      2022     299.0

54

# Initialization

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



def create_lag_features(data, lag_steps=1, group=["Major"], target="1st_Year"):
    if group:
        for i in range(1, lag_steps + 1):
            data[f'{target}_lag_{i}'] = data.groupby(group)[target].shift(i)
    else:
        for i in range(1, lag_steps + 1):
            data[f'{target}_lag_{i}'] = data[target].shift(i)
    return data

def create_rolling_mean(data, group=["Major"], window_size=3, target="1st_Year", min_periods=1, lag_steps=0):
    lag_string = f'_lag_{lag_steps}' if lag_steps else ""
    if group:
        data[f'{target}_rolling_mean{lag_string}'] = data.groupby(group)[target].shift(lag_steps).rolling(window=window_size, min_periods=min_periods).mean().values
    else:
        data[f'{target}_rolling_mean{lag_string}'] = data[target].shift(lag_steps).rolling(window=window_size, min_periods=min_periods).mean()
    return data

def create_rolling_std(data, group=["Major"], window_size=3, target="1st_Year", min_periods=1, lag_steps=0):
    lag_string = f'_lag_{lag_steps}' if lag_steps else ""
    if group:
        data[f'{target}_rolling_std{lag_string}'] = data.groupby(group)[target].shift(lag_steps).rolling(window=window_size, min_periods=min_periods).std().values
    else:
        data[f'{target}_rolling_std{lag_string}'] = data[target].shift(lag_steps).rolling(window=window_size, min_periods=min_periods).std()
    return data




# SMAPE
def smape(y_true, y_pred):
    return np.mean(2 * np.abs((y_true - y_pred) /  (np.abs(y_true) + np.abs(y_pred)))) * 100

# MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) /  y_true)) * 100


def modelresults(y_true, predictions):
    mae = mean_absolute_error(y_true, predictions)
    rmse = np.sqrt(mean_squared_error(y_true, predictions))
    r2 = r2_score(y_true, predictions)
    smape_score = smape(y_true, predictions)
    mape_score = mape(y_true, predictions)

    print(f"Symmetric Mean Absolute Percentage Error: {smape_score:.2f}%")
    print(f'Mean Absolute Percentage Error: {mape_score:.2f}%')
    print(f'Mean Absolute Error: {mae:.4f}')
    print(f'Root Mean Squared Error: {rmse:.4f}')
    print(f'R2 Score: {r2*100:.4f}')

    return {
        "smape": smape_score,
        "mape": mape_score,
        "mae": mae,
        "rmse": rmse,
        "r2": r2
    }


# Determine the start of enrollment period
def determine_start_month(semester):
    if semester == 1:
       return 5  # June
    else:
        
        return 10  # November

# User Authentication

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.sql import func

Base = declarative_base()

# Define the association table
user_roles = Table('user_roles', Base.metadata,
    Column('user_id', Integer, ForeignKey('users.id')),
    Column('role_id', Integer, ForeignKey('roles.id'))
)

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    password_hash = Column(String(255), nullable=False)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())
    login_counter = Column(Integer, default=0)


    roles = relationship('Role', secondary=user_roles, back_populates='users')

class Role(Base):
    __tablename__ = 'roles'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), unique=True, nullable=False)
    description = Column(Text)

    users = relationship('User', secondary=user_roles, back_populates='roles')

# Create the tables
Base.metadata.create_all(engine)

print("Tables for users, roles, and user_roles have been created.")

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Add Admin and User roles
admin_role = Role(name='Admin', description='Administrator role with full access')
user_role = Role(name='User', description='Standard user role with limited access')

# Check if roles already exist
existing_admin = session.query(Role).filter_by(name='Admin').first()
existing_user = session.query(Role).filter_by(name='User').first()

if not existing_admin:
    session.add(admin_role)
    print("Admin role added.")
else:
    print("Admin role already exists.")

if not existing_user:
    session.add(user_role)
    print("User role added.")
else:
    print("User role already exists.")

# Commit the changes
session.commit()

# Close the session
session.close()

print("Admin and User roles have been added to the database.")

D:\Local\temp\ipykernel_8108\2511560472.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Tables for users, roles, and user_roles have been created.
Admin role added.
User role added.
Admin and User roles have been added to the database.


In [4]:
%pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [9]:
import bcrypt
# Admin user details
username = "Admin"
email = "admin@hau.edu.ph"
password = "@ngeliteF0reca$t4dmin"  # Replace with the actual password you want to use

# Hash the password
hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt(rounds=10))

try:
    # Check if the user already exists
    existing_user = session.execute(text("SELECT * FROM users WHERE username = :username OR email = :email"),
                                    {"username": username, "email": email}).fetchone()
    
    if existing_user:
        # Update the existing user
        session.execute(text("""
            UPDATE users 
            SET username = :username, email = :email, password_hash = :password_hash
            WHERE id = :user_id
        """), {
            "username": username,
            "email": email,
            "password_hash": hashed_password.decode('utf-8'),
            "user_id": existing_user[0]
        })
        print("Admin user updated successfully.")
    else:
        # Insert the new admin user
        session.execute(text("""
            INSERT INTO users (username, email, password_hash)
            VALUES (:username, :email, :password_hash)
        """), {
            "username": username,
            "email": email,
            "password_hash": hashed_password.decode('utf-8')  # Convert bytes to string
        })
        
        # Get the user id of the newly inserted admin
        user_id = session.execute(text("SELECT id FROM users WHERE username = :username"),
                                  {"username": username}).scalar()
        
        # Insert the admin role for this user
        session.execute(text("""
            INSERT INTO user_roles (user_id, role_id)
            VALUES (:user_id, (SELECT id FROM roles WHERE name = 'Admin'))
        """), {
            "user_id": user_id
        })
        
        session.commit()
        print("Admin user added successfully.")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()



Admin user updated successfully.


# Major Data

In [7]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func


# Create a base class for declarative models
Base = declarative_base()

# Define the Major model
class Major(Base):
    __tablename__ = 'majors'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)
    department = Column(String(50), nullable=False)
    latest_year = Column(Integer)  # New column for the latest year
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())

    def __repr__(self):
        return f"<Major(name='{self.name}', department='{self.department}', latest_year={self.latest_year})>"

# Create the table in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# # Example: Add a new major
# new_major = Major(name="Computer Science", department="School of Computing")
# session.add(new_major)
# session.commit()

# # Example: Query all majors
# majors = session.query(Major).all()
# for major in majors:
#     print(major)

# # Don't forget to close the session when you're done
# session.close()

# Update Major Data

In [6]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import pandas as pd

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

def get_college_data(df):
    df = df[~df['Department'].isin(['GS', 'JHS', 'SHS', 'HAUSPELL', 'HAU', 'MA'])]
    df = df[~df['Major'].isin(['TOTAL', 'GRAND TOTAL'])]
    df = df.drop(df.loc[:, "Grade_1":"Grade_12"].columns, axis=1)
    df = df.reset_index(drop=True)
    return df

def clean_college_data(df):
    # Step 1: Replace similar majors
    for old_majors, new_major in similar_majors_dict.items():
        df['Major'] = df['Major'].replace(old_majors, new_major[0])
     
    # Step 4: Drop unwanted majors
    df = df[~df['Major'].isin(drop_majors)]
        
    # Step 6: Combine majors with the same name within the same semester
    df = df.groupby(['Start_Year', 'Semester',
                     'Major', 'Department'], as_index=False).sum(["1st_Year", "2nd_Year", "3rd_Year",
                                                                  "4th_Year", "5th_Year", "TOTAL"])
    return df

# Load the enrollment data
enrollment_df = pd.read_sql_table('enrollment', engine)

# Apply the get_college_data function
college_data = get_college_data(enrollment_df)
college_data = clean_college_data(college_data)

# Get unique combinations of Major and Department with the latest year
unique_majors = college_data.groupby(['Major', 'Department'])['Start_Year'].max().reset_index()

# Get the list of existing majors from the database
existing_majors = session.query(Major.name, Major.latest_year).all()
existing_majors_dict = {major[0]: major[1] for major in existing_majors}

# Add or update each unique Major-Department combination in the majors table
new_majors = []
updated_majors = []
for _, row in unique_majors.iterrows():
    if row['Major'] not in existing_majors_dict:
        new_majors.append(Major(name=row['Major'], department=row['Department'], latest_year=row['Start_Year']))
        print(f"Adding new major: {row['Major']} - {row['Department']} (Year: {row['Start_Year']})")
    elif row['Start_Year'] > existing_majors_dict[row['Major']]:
        major = session.query(Major).filter_by(name=row['Major']).first()
        major.latest_year = row['Start_Year']
        updated_majors.append(major)
        print(f"Updating major: {row['Major']} - New latest year: {row['Start_Year']}")
    else:
        print(f"Major already up to date: {row['Major']} (Year: {existing_majors_dict[row['Major']]})")

# Bulk insert new majors and update existing ones
if new_majors:
    session.bulk_save_objects(new_majors)
    print(f"Added {len(new_majors)} new majors to the database.")
if updated_majors:
    session.bulk_save_objects(updated_majors, update_changed_only=True)
    print(f"Updated {len(updated_majors)} existing majors in the database.")

# Commit the changes
session.commit()

# Close the session
session.close()

print("Major update process completed.")

NameError: name 'Major' is not defined

# Processed Data

In [6]:
import pandas as pd
import datetime

# Get the current year
current_year = datetime.datetime.now().year

# Load data
df = pd.read_csv("data/Enrollment_Data.csv")
cpi_df = pd.read_csv("data/CPI_Education.csv")
inflation_df = pd.read_csv("data/Inflation_Rate.csv")
admission_df = pd.read_csv("data/Admission_Data.csv")
hfce_df = pd.read_csv("data/HFCE.csv")

df = clean_data(df)
# Basic preprocessing
df = df.drop(columns=["End_Year"])

# Function to add placeholder for current year
def add_current_year_placeholder(df, year_column, value_columns):
    max_year = df[year_column].max()
    if max_year < current_year:
        placeholder = df[df[year_column] == max_year].copy()
        placeholder[year_column] = current_year
        for col in value_columns:
            placeholder[col] = float('nan')
        df = pd.concat([df, placeholder], ignore_index=True)
    return df.sort_values(by=[year_column]).reset_index(drop=True)

# CPI data processing
cpi_df_copy = cpi_df.copy()
cpi_df_copy["Month"] = cpi_df["Month"].map({
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12,
})
cpi_df_copy = cpi_df_copy.dropna()
cpi_df_copy["Year"] = cpi_df_copy["Year"].astype(int)
cpi_df_copy = cpi_df_copy.groupby('Year')['CPI_Region3'].mean().reset_index()
cpi_df_copy = add_current_year_placeholder(cpi_df_copy, "Year", ["CPI_Region3"])

# Inflation data processing
inflation_df = inflation_df[["Start_Year", "Inflation_Rate"]].dropna()

inflation_df = add_current_year_placeholder(inflation_df, "Start_Year", ["Inflation_Rate"])
inflation_df = create_lag_features(inflation_df, group=None, target="Inflation_Rate", lag_steps=1)
inflation_df = inflation_df.drop(columns=["Inflation_Rate"])

# HFCE data processing
hfce_df = hfce_df.groupby('Start_Year').mean().reset_index()
hfce_df = add_current_year_placeholder(hfce_df, "Start_Year", ["HFCE_Education", "HFCE"])

# Admission data processing
admission_df = admission_df.drop(columns=["Number_of_Processed_Applicants", "Number_of_Enrolled_Applicants"])
admission_df["Department"] = admission_df["Department"].replace("CICT", "SOC")
admission_df = add_current_year_placeholder(admission_df, "Start_Year", ["Number_of_Applicants"])

# Basic enrollment data processing
df = df.groupby(['Start_Year', 'Semester', 'Major', 'Department'], as_index=False).sum(["1st_Year", "2nd_Year", "3rd_Year", "4th_Year", "5th_Year", "Grade_12", "TOTAL"])
df = df[~df['Department'].isin(['GS', 'JHS', 'SHS', 'HAUSPELL', 'HAU', 'MA'])]
df = df[~df['Major'].isin(['TOTAL', 'GRAND TOTAL'])]
df = df.drop(df.loc[:, "Grade_1":"Grade_11"].columns, axis=1)
df = df.reset_index(drop=True)

# Apply the function to determine start month
df['Start_Month'] = df['Semester'].apply(determine_start_month)

# Convert numerical features to int
numerical_features = df.drop(columns=["Major", "Department"]).columns
df[numerical_features] = df[numerical_features].astype(int)

# Merging data
df_encoded = df.copy()
df_encoded = df_encoded.merge(cpi_df_copy, left_on=["Start_Year"], right_on=["Year"], how="left")
df_encoded = df_encoded.merge(inflation_df, on=["Start_Year"], how="left")
df_encoded = df_encoded.merge(admission_df, on=["Department", "Start_Year"], how="left")
df_encoded = df_encoded.merge(hfce_df, on=["Start_Year"], how="left")
df_encoded = df_encoded.drop(columns=["Quarter", "Year"])

# Sort the DataFrame
df_encoded = df_encoded.sort_values(by=['Start_Year', 'Start_Month'])


In [8]:
df_encoded.columns

Index(['Start_Year', 'Semester', 'Major', 'Department', '1st_Year', '2nd_Year',
       '3rd_Year', '4th_Year', '5th_Year', 'Grade_12', 'TOTAL', 'Start_Month',
       'CPI_Region3', 'Inflation_Rate_lag_1', 'Number_of_Applicants',
       'HFCE_Education', 'HFCE'],
      dtype='object')

In [7]:
# Create the table and insert the data
df_encoded.to_sql('processed_factors', engine, if_exists='replace', index=False)

print("Processed data has been saved to the 'processed_data' table in the database.")

Processed data has been saved to the 'processed_data' table in the database.


## OLD

In [3]:
import joblib
from sklearn.preprocessing import OneHotEncoder


df = pd.read_csv("data/Enrollment_Data.csv")
cpi_df = pd.read_csv("data/CPI_Education.csv")
inflation_df = pd.read_csv("data/Inflation_Rate.csv")
admission_df = pd.read_csv("data/Admission_Data.csv")
hfce_df = pd.read_csv("data/HFCE.csv")

df = df.drop(columns=["End_Year"])

admission_df = admission_df.drop(columns=["Number_of_Processed_Applicants", "Number_of_Enrolled_Applicants"])
admission_df["Department"] = admission_df["Department"].replace("CICT", "SOC")

# Get Exponential Moving Average for Admission Number of Applicants
# Filter the 2023 data
admission_2023 = admission_df[admission_df["Start_Year"] == 2023].copy()

# Update the Start_Year to 2024
admission_2023["Start_Year"] = 2024

# Set the Number_of_Applicants values to NaN for the 2024 data
admission_2023["Number_of_Applicants"] = float('nan')

# Append the new 2024 data to the original admission_df
admission_df = pd.concat([admission_df, admission_2023], ignore_index=True)

# Ensure the DataFrame is sorted by 'Start_Year' and reset the index
admission_df = admission_df.sort_values(by=['Start_Year']).reset_index(drop=True)

admission_df = admission_df[admission_df["Start_Year"] <= 2024]
admission_df = create_lag_features(admission_df, group=["Department"], target="Number_of_Applicants", lag_steps=1)
# admission_df = create_rolling_mean(admission_df, group=["Department"], target="Number_of_Applicants", window_size=3, min_periods=1)
admission_df = create_rolling_std(admission_df, group=["Department"], target="Number_of_Applicants", window_size=3, min_periods=1, lag_steps=0)
# admission_df = create_lag_features(admission_df, group=["Department"], target="Number_of_Applicants", window_size=3, min_periods=1, lag_steps=1)

admission_df = admission_df.fillna(0)

cpi_df_copy = cpi_df.copy()
# cpi_df_copy = cpi_df_copy[cpi_df_copy["Year"] < 2024]
cpi_df_copy["Month"] = cpi_df["Month"].map({
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12,
})
cpi_df_copy = cpi_df_copy.dropna()
cpi_df_copy[["Year"]] = cpi_df_copy[["Year"]].astype(int)
cpi_df_copy = cpi_df_copy.groupby('Year')['CPI_Region3'].mean().reset_index()
cpi_df_copy = create_rolling_std(cpi_df_copy, group=None, target="CPI_Region3", window_size=6, lag_steps=0)

# Assuming df is your DataFrame

# Step 1: Replace similar majors
for old_majors, new_major  in similar_majors_dict.items():
    df['Major'] = df['Major'].replace(old_majors, new_major[0])

# Step 2: Correct departments

for major, department in incorrect_department_dict.items():
    df.loc[df["Major"]== major, 'Department'] = department
    
# Step 3: Rename departments
df['Department'] = df['Department'].replace(department_dict)

# Step 4: Drop unwanted majors
df = df[~df['Major'].isin(drop_majors)]

# # Step 5: Filter by threshold
# major_counts = df['Major'].value_counts()
# valid_majors = major_counts[major_counts >= threshold].index
# df = df[finalDf['Major'].isin(valid_majors)]

# Step 6: Combine majors with the same name within the same semester
df = df.groupby(['Start_Year', 'Semester', 'Major', 'Department'], as_index=False).sum(["1st_Year", "2nd_Year", "3rd_Year", "4th_Year", "5th_Year", "Grade_12", "TOTAL"])


shs_df = df[df["Department"] == "SHS"].pivot_table(index=["Start_Year", "Semester"], columns="Major", values="Grade_12").reset_index().fillna(0)
shs_df["Start_Year"] += 1
shs_df = shs_df.rename(columns={col: f"{col}_lag_1" for col in shs_df.columns.drop(["Start_Year", "Semester"])})



df = df[~df['Department'].isin(['GS', 'JHS', 'SHS', 'HAUSPELL', 'HAU', 'MA'])]
df = df[~df['Major'].isin(['TOTAL', 'GRAND TOTAL'])]
df = df.drop(df.loc[:, "Grade_1":"Grade_11"].columns, axis=1)
df = df.reset_index(drop=True)





inflation_df = inflation_df[["Start_Year", "Inflation_Rate"]]
inflation_df_copy = inflation_df.dropna()
inflation_2023 = inflation_df_copy[inflation_df_copy["Start_Year"] == 2023].copy()

# Update the Start_Year to 2024
inflation_2023["Start_Year"] = 2024

# Set the Number_of_Applicants values to NaN for the 2024 data
inflation_2023["Inflation_Rate"] = float('nan')

# Append the new 2024 data to the original admission_df
inflation_df_copy = pd.concat([inflation_df_copy, inflation_2023], ignore_index=True)

# Ensure the DataFrame is sorted by 'Start_Year' and reset the index
inflation_df_copy = inflation_df_copy.sort_values(by=['Start_Year']).reset_index(drop=True)

inflation_df_copy = inflation_df_copy[inflation_df_copy["Start_Year"] <= 2024]

# inflation_df_copy = create_lag_features(cpi_df_copy, group=None, target="Inflation_Rate", lag_steps=1)
# inflation_df_copy = create_rolling_mean(cpi_df_copy, group=None, target="Inflation_Rate", window_size=6)
inflation_df_copy = create_rolling_std(inflation_df_copy, group=None, target="Inflation_Rate", window_size=3, lag_steps=1)
inflation_df_copy = inflation_df_copy.drop(columns=["Inflation_Rate"])


# HFCE data processing
hfce_df = hfce_df.groupby('Start_Year').mean().reset_index()
hfce_df_copy = hfce_df.dropna()
hfce_df_copy = create_lag_features(hfce_df_copy, group=None, target="HFCE_Education", lag_steps=1)
hfce_df_copy = create_rolling_std(hfce_df_copy, group=None, target="HFCE_Education", window_size=6)
hfce_df_copy = create_lag_features(hfce_df_copy, group=None, target="HFCE", lag_steps=1)
hfce_df_copy = create_rolling_std(hfce_df_copy, group=None, target="HFCE", window_size=6)
hfce_df_copy = hfce_df_copy.drop(columns=["HFCE_Education", "HFCE"])




# Apply the function and create Semester_Start column
df['Start_Month'] = df['Semester'].apply(determine_start_month)

# Get End Month after 18 weeks
# df['End_Month'] = df['Start_Month'] + 4
df = df.fillna(0)



# # Add 'Modified' column where 1st_Year is 0
# df['Modified'] = df['1st_Year'].apply(lambda x: True if x == 0 else False)


numerical_features =  df.drop(columns=["Major", "Department"]).columns
df[numerical_features] = df[numerical_features].astype(int)


# Add CPI_Region3 to the dataframe
df_encoded = df.copy()
df_encoded["Start_Year"].unique()


# df_encoded = df.copy()
# df_encoded.drop(drop_majors, axis=1, inplace=True)


# Merging data
df_encoded = df_encoded.merge(cpi_df_copy, left_on=["Start_Year"], right_on=["Year"], how="left")
df_encoded = df_encoded.merge(inflation_df_copy, on=["Start_Year"], how="left")
df_encoded = df_encoded.merge(admission_df, on=["Department", "Start_Year"], how="left")
df_encoded = df_encoded.merge(hfce_df_copy, on=["Start_Year"], how="left")
df_encoded = df_encoded.drop(columns=["Year"])


# Lagged Features
# Calculate Exponential Moving Average (EMA)
ema_df = pd.DataFrame()
# ema_df['Inflation_Rate_EMA'] = df_encoded.groupby(['Start_Year', 'Major'])['Inflation_Rate'].transform(lambda x: x.ewm(span=2, adjust=False).mean())

# Ensure DataFrame is sorted by 'Start_Year' and index
df_encoded.sort_values(by=['Start_Year', 'Semester'], inplace=True)

df_encoded = create_lag_features(df_encoded, lag_steps=1)
# df_encoded = create_rolling_mean(df_encoded, lag_steps=1, window_size=3)
df_encoded = create_rolling_std(df_encoded, lag_steps=1, window_size=3)

# df_encoded = create_lag_features(df_encoded, lag_steps=2, target="Grade_12")
# df_encoded = create_rolling_std(df_encoded, lag_steps=2, window_size=3, target="Grade_12")

df_encoded = create_lag_features(df_encoded, lag_steps=1, target="TOTAL")

# df_encoded = create_rolling_std(df_encoded, window_size=3, target="2nd_Year")
# df_encoded = create_rolling_std(df_encoded, window_size=3, target="3rd_Year")
# df_encoded = create_rolling_std(df_encoded, window_size=3, target="4th_Year")



# df_encoded = df_encoded.drop(columns=["Number_of_Applicants", "CPI_Region3"])



# ema_df['1st_Year_EMA'] = df_encoded.groupby('Major')['1st_Year'].transform(lambda x: x.ewm(span=2, adjust=False).mean())
# ema_df['2nd_Year_EMA'] = df_encoded.groupby('Major')['2nd_Year'].transform(lambda x: x.ewm(span=2, adjust=False).mean())
# ema_df['3rd_Year_EMA'] = df_encoded.groupby('Major')['3rd_Year'].transform(lambda x: x.ewm(span=2, adjust=False).mean())
# ema_df['4th_Year_EMA'] = df_encoded.groupby('Major')['4th_Year'].transform(lambda x: x.ewm(span=2, adjust=False).mean())

#df_encoded = df_encoded.dropna()


# Assume df_train is your training dataframe
# One-hot encode departments and majors
dept_encoder = OneHotEncoder(sparse_output=False)
major_encoder = OneHotEncoder(sparse_output=False)

dept_encoded = dept_encoder.fit_transform(df_encoded[['Department']])
major_encoded = major_encoder.fit_transform(df_encoded[['Major']])

# Perform PCA
dept_pca = PCA(n_components=2, random_state=24)
major_pca = PCA(n_components=2, random_state=24)

department_principalComponents = dept_pca.fit_transform(dept_encoded)
department_principalDf = pd.DataFrame(data = department_principalComponents, columns = [f"Department_PC{i+1}" for i in range(2)])

major_principalComponents = major_pca.fit_transform(major_encoded)
major_principalDf = pd.DataFrame(data = major_principalComponents, columns = [f"Major_PC{i+1}" for i in range(2)])


# Save the encoders and PCA objects
joblib.dump(dept_encoder, 'data/dept_encoder.pkl')
joblib.dump(major_encoder, 'data/major_encoder.pkl')
joblib.dump(dept_pca, 'data/dept_pca.pkl')
joblib.dump(major_pca, 'data/major_pca.pkl')


# department_df = pd.get_dummies(df_encoded[["Department"]])
# major_df = pd.get_dummies(df_encoded[["Major"]])
# # df_encoded = pd.concat([df_encoded, major_df], axis=1)


# # Dimensionality Reduction for Major
# pca = PCA(n_components=2, random_state=24)
# department_principalComponents = pca.fit_transform(department_df)
# department_principalDf = pd.DataFrame(data = department_principalComponents, columns = [f"Department_PC{i+1}" for i in range(2)])

# major_principalComponents = pca.fit_transform(major_df)
# major_principalDf = pd.DataFrame(data = major_principalComponents, columns = [f"Major_PC{i+1}" for i in range(2)])


finalDf = pd.concat([df_encoded, department_principalDf, major_principalDf], axis = 1)

# finalDf = finalDf.dropna()

# Drop major with less than 10 students overall
major_counts = finalDf.groupby('Major')['1st_Year'].sum()
valid_majors = major_counts[major_counts >= 10].index
finalDf = finalDf[finalDf['Major'].isin(valid_majors)]

finalDf = finalDf.merge(shs_df, on=["Start_Year", "Semester"], how="left")
finalDf = finalDf.drop(columns=["CPI_Region3", "Number_of_Applicants"])


# Step 1: Group by year and major to get the sum of students in each major for each year
grouped = finalDf.groupby(['Start_Year', 'Semester', 'Major'])['1st_Year_lag_1'].sum().reset_index()

# Step 2: Calculate the total number of students for each year
total_students_per_year = grouped.groupby(['Start_Year', 'Semester'])['1st_Year_lag_1'].sum().reset_index()
total_students_per_year.rename(columns={'1st_Year_lag_1': 'Total_1st_Year_Students_lag_1'}, inplace=True)

# Step 3: Merge the total students per year with the grouped data
distribution_df = pd.merge(grouped, total_students_per_year, on=['Start_Year', 'Semester'])

# Step 4: Calculate the percentage distribution of each major
distribution_df['Percentage_Distribution_lag_1'] = (distribution_df['1st_Year_lag_1'] / distribution_df['Total_1st_Year_Students_lag_1']) * 100


finalDf = finalDf.merge(distribution_df.drop(columns=["1st_Year_lag_1"]), on=['Start_Year', 'Semester', 'Major'])


# # Create multiple columns filled with 0s
# for i in range(3, 5):
#     finalDf[f"Grade_12_lag_{i}"] = 1
    

finalDf = finalDf.sort_values(by=['Start_Year', 'Start_Month'])

In [8]:
finalDf_copy = finalDf.fillna(0).drop_duplicates()
finalDf_copy.to_sql('processed_data', engine, if_exists='replace', index=False)


747

# CAUTION
DO NOT RUN THIS CODE

In [ ]:
# Throw error to avoid run all to here
raise Exception("Execution stopped to prevent running the irreversible code.")


## Clear majors Table

In [1]:
from sqlalchemy import text

# Create a new session
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Delete all rows from the majors table
    session.execute(text("DELETE FROM majors"))
    
    # Commit the transaction
    session.commit()
    print("All rows have been deleted from the majors table.")
except Exception as e:
    # If an error occurs, rollback the changes
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Close the session
    session.close()

NameError: name 'sessionmaker' is not defined

## Delete majors Table

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create a MetaData instance
metadata = MetaData()

# Reflect the existing tables
metadata.reflect(bind=engine)


try:
    # Check if the majors table exists
    if 'majors' in metadata.tables:
        # Drop the majors table
        metadata.tables['majors'].drop(engine)
        print("The 'majors' table has been successfully dropped.")
    else:
        print("The 'majors' table does not exist.")
    
    # Commit the transaction
    session.commit()
except Exception as e:
    # If an error occurs, rollback the changes
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Close the session
    session.close()

print("Table deletion process completed.")

The 'majors' table has been successfully dropped.
Table deletion process completed.


## Delete processed_data Table


In [12]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

# Create a MetaData instance
metadata = MetaData()

# Reflect the existing tables
metadata.reflect(bind=engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Check if the processed_data table exists
    if 'processed_data' in metadata.tables:
        # Drop the processed_data table
        metadata.tables['processed_data'].drop(engine)
        print("The 'processed_data' table has been successfully dropped.")
    else:
        print("The 'processed_data' table does not exist.")
    
    # Commit the transaction
    session.commit()
except Exception as e:
    # If an error occurs, rollback the changes
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Close the session
    session.close()

print("Table deletion process completed.")


The 'processed_data' table does not exist.
Table deletion process completed.
